In [13]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path if not already added
if project_root not in sys.path:
    sys.path.append(project_root)

In [19]:
from src.functions.data_generation import DataGeneration
from sklearn.model_selection import train_test_split

data = DataGeneration.generate_data(
    n_samples=10000,
    n_features=20,
    n_informative=4,
    relationship='friedman1',
    noise=0.1,
    random_state=2319
)

# Draw bootstrap sample
bootstrap_data = data.sample(n=1000, replace=True, random_state=89321)
bootstrap_data = DataGeneration.add_epsilon(bootstrap_data)

display(bootstrap_data)

X = bootstrap_data.drop(columns=["Target"])
y = bootstrap_data["Target"]

# Split data
X_proper, X_test, y_proper, y_test = train_test_split(X, y, test_size=0.5)

        
def to_numpy_safe(x):
    return x.to_numpy() if hasattr(x, "to_numpy") else x

# Convert to numpy arrays if needed
X_proper = to_numpy_safe(X_proper)
y_proper = to_numpy_safe(y_proper)
X_test   = to_numpy_safe(X_test)
y_test   = to_numpy_safe(y_test)

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Target
1907,0.253523,0.825448,0.675537,0.285195,0.854191,0.587814,0.145865,0.986894,0.597206,0.294292,...,0.461627,0.188441,0.744561,0.265578,0.114102,0.849126,0.825639,0.005954,0.635366,0.454211
1836,0.305088,0.910548,0.143417,0.081087,0.949853,0.757272,0.618384,0.008670,0.986418,0.769552,...,0.369754,0.388349,0.199190,0.666208,0.145463,0.121171,0.747518,0.646308,0.558050,0.528995
9843,0.011580,0.282816,0.991478,0.853351,0.024669,0.525313,0.080781,0.838431,0.123227,0.731516,...,0.687053,0.960579,0.483801,0.198679,0.422602,0.407256,0.164494,0.049819,0.444398,0.449629
5340,0.308421,0.869729,0.081287,0.790356,0.933519,0.210263,0.046326,0.168196,0.572453,0.101654,...,0.013507,0.507834,0.658442,0.900163,0.291200,0.655146,0.578647,0.737143,0.870682,0.793156
7901,0.436253,0.117922,0.425866,0.371658,0.105172,0.057389,0.981601,0.577723,0.674280,0.320127,...,0.649198,0.181603,0.784824,0.947418,0.812414,0.056599,0.776566,0.231559,0.335934,0.180458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3917,0.843929,0.257567,0.093888,0.752753,0.197412,0.217975,0.313634,0.052515,0.919847,0.422588,...,0.464023,0.693488,0.156952,0.810106,0.865028,0.951747,0.155684,0.681338,0.342824,0.593861
2854,0.254809,0.597354,0.022223,0.770603,0.319848,0.019483,0.311217,0.641004,0.086016,0.082985,...,0.403210,0.492664,0.482588,0.226144,0.213515,0.619329,0.037284,0.927696,0.534298,0.622204
1080,0.084180,0.421865,0.194716,0.521149,0.954720,0.367483,0.835111,0.719456,0.853305,0.154741,...,0.310830,0.679120,0.207506,0.597039,0.274063,0.188937,0.260290,0.008697,0.610755,0.424123
8401,0.636961,0.199853,0.021698,0.251745,0.103381,0.087120,0.849290,0.993439,0.699922,0.695631,...,0.621985,0.533924,0.046247,0.275752,0.171159,0.175730,0.553808,0.339822,0.335365,0.373900


In [22]:
from online_cp.CPS import NearestNeighboursPredictionMachine
from online_cp.CPS import RidgePredictionMachine, KernelRidgePredictionMachine
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# cps = NearestNeighboursPredictionMachine(k=5)
# cps = RidgePredictionMachine(autotune=True)
kernel = C(0.001) * RBF(length_scale=0.001)
cps = KernelRidgePredictionMachine(a=0.01, kernel=kernel)

cps.learn_initial_training_set(X_proper, y_proper)


# x = [7.34768661e+01, 
#      3.07355747e+02, 
#      9.98483471e-01, 
#      9.49902854e+00,
#      3.62115400e-01, 
#      5.48508689e-01, 
#      8.40112768e-01, 
#      3.42234629e-01,
#      3.44227224e-01, 
#      7.71883283e-01, 
#      2.42761017e-01, 
#      3.44293874e-01,
#      5.20936857e-01, 
#      5.20083931e-01, 
#      8.81639709e-01, 
#      3.41171486e-01,
#      1.60554105e-01, 
#      5.98680164e-01, 
#      7.88717442e-01, 
#      7.44306979e-02]

for i, (x, y) in enumerate(zip(X_test, y_test)):
    #try:

    cpd, precomputed = cps.predict_cpd(x=x, return_update=True)
    
    # except Exception as e:
    #     print(i)
    #     print()
    #     print("Exception occurred for x =", x)
    #     print()
    #     print("Error:", e)
    